# Взаимодействие с Марусей

Magic функция, которая будет одновременно выполняет код ячейки и экспортирует его в файл, так как встроенная функция %%writefile такой функциональности не поддерживает.

In [1]:
from IPython.core.magic import register_cell_magic


@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = "w"
    if len(argz) == 2 and argz[0] == "-a":
        mode = "a"
    with open(file, mode) as f:
        f.write(cell + "\n\n")
    get_ipython().run_cell(cell)

Секция импорта

In [2]:
%%write_and_run front_mail.py
import json

import boto3

Получаем данные для ответа

In [3]:
%%write_and_run -a front_mail.py
# make connection to s3
session = boto3.session.Session()
s3 = session.client(service_name="s3", endpoint_url="https://storage.yandexcloud.net")

In [4]:
%%write_and_run -a front_mail.py
# get data
get_object_response = s3.get_object(Bucket="scrapper", Key="habr/news")
response_body = get_object_response["Body"].read()

In [5]:
%%write_and_run -a front_mail.py
# parse data
response_parsed = response_body.decode("utf-8").split("\n")

In [6]:
response_parsed

['Издание BleepingComputer опубликовало подробности об атаке вируса-вымогателя на Garmin.',
 'США планируют отправить маленькие ядерные реакторы 10 кВт на Луну и Марс.',
 'Пользователь обнаружил, что приложение Instagram в iOS 14 нештатно использует камеру. Разработчик пояснил, что это баг.',
 "Неизвестный мститель саботирует работу ботнета Emotet, заменяя вредоносную нагрузку анимированными GIF'ками.",
 'В Minecraft теперь можно загрузить Windows 95 и поиграть в Doom.',
 'Microsoft будет принудительно устанавливать обновление May 2020 Update на ПК с Windows 10 версий 1809 и 1903.',
 'Пользователи термостатов Nest жалуются на сбои в работе Wi-Fi модуля, Google пытается решить проблему с ноября 2019 года.',
 'Обучающие видео иранских хакеров утекли в сеть.',
 'Новости Blender3d.',
 'Затраты на переход на отечественный софт в банках оценили в сотни миллиардов рублей.',
 'Сервисы и устройства Garmin слегли, вероятно, из-за атаки вируса-вымогателя.',
 'Amazon встречается со стартапами, а п

In [7]:
%%write_and_run -a front_mail.py
# limit data for response
size = 0
text_body = []
for header in response_parsed:
    size += len(header) + len("\n")
    if size > 1024:
        break
    text_body.append(header)

In [8]:
"\n".join(text_body)

"Издание BleepingComputer опубликовало подробности об атаке вируса-вымогателя на Garmin.\nСША планируют отправить маленькие ядерные реакторы 10 кВт на Луну и Марс.\nПользователь обнаружил, что приложение Instagram в iOS 14 нештатно использует камеру. Разработчик пояснил, что это баг.\nНеизвестный мститель саботирует работу ботнета Emotet, заменяя вредоносную нагрузку анимированными GIF'ками.\nВ Minecraft теперь можно загрузить Windows 95 и поиграть в Doom.\nMicrosoft будет принудительно устанавливать обновление May 2020 Update на ПК с Windows 10 версий 1809 и 1903.\nПользователи термостатов Nest жалуются на сбои в работе Wi-Fi модуля, Google пытается решить проблему с ноября 2019 года.\nОбучающие видео иранских хакеров утекли в сеть.\nНовости Blender3d.\nЗатраты на переход на отечественный софт в банках оценили в сотни миллиардов рублей.\nСервисы и устройства Garmin слегли, вероятно, из-за атаки вируса-вымогателя.\nAmazon встречается со стартапами, а потом выпускает конкурирующие проду

Формируем ответ для Маруси

In [11]:
%%write_and_run -a front_mail.py
response = json.dumps(
    {
        "statusCode": 200,
        "headers": {"Content-Type": "application/json"},
        "isBase64Encoded": False,
        "body": {
            "response": {"text": "\n".join(text_body), "end_session": True},
            "session": {
                "session_id": "574d41e0-a41e-4028-a73a-6f5b5bfed299",
                "message_id": 0,
                "user_id": "6953b29afe19e372ecdd34d07b3eae3c2f69b9f04e8cb15e157c4a9e056d58ee",
            },
            "version": "1.0",
        },
    }
)

In [12]:
response

'{"statusCode": 200, "headers": {"Content-Type": "application/json"}, "isBase64Encoded": false, "body": {"response": {"text": "\\u0418\\u0437\\u0434\\u0430\\u043d\\u0438\\u0435 BleepingComputer \\u043e\\u043f\\u0443\\u0431\\u043b\\u0438\\u043a\\u043e\\u0432\\u0430\\u043b\\u043e \\u043f\\u043e\\u0434\\u0440\\u043e\\u0431\\u043d\\u043e\\u0441\\u0442\\u0438 \\u043e\\u0431 \\u0430\\u0442\\u0430\\u043a\\u0435 \\u0432\\u0438\\u0440\\u0443\\u0441\\u0430-\\u0432\\u044b\\u043c\\u043e\\u0433\\u0430\\u0442\\u0435\\u043b\\u044f \\u043d\\u0430 Garmin.\\n\\u0421\\u0428\\u0410 \\u043f\\u043b\\u0430\\u043d\\u0438\\u0440\\u0443\\u044e\\u0442 \\u043e\\u0442\\u043f\\u0440\\u0430\\u0432\\u0438\\u0442\\u044c \\u043c\\u0430\\u043b\\u0435\\u043d\\u044c\\u043a\\u0438\\u0435 \\u044f\\u0434\\u0435\\u0440\\u043d\\u044b\\u0435 \\u0440\\u0435\\u0430\\u043a\\u0442\\u043e\\u0440\\u044b 10 \\u043a\\u0412\\u0442 \\u043d\\u0430 \\u041b\\u0443\\u043d\\u0443 \\u0438 \\u041c\\u0430\\u0440\\u0441.\\n\\u041f\\u043e\\u043b\\